<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar2/notebooks/oscar_test_make_auto_transforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Latest Code From Team's GitHub

In [1]:
!git clone -b auto_transform "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 401 (delta 73), reused 69 (delta 47), pack-reused 282
Receiving objects: 100% (401/401), 107.87 KiB | 488.00 KiB/s, done.
Resolving deltas: 100% (262/262), done.


In [2]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

In [3]:
!ls

csgy6953_DeepLearning_Midterm  sample_data  src


In [4]:
!cat src/transforms.py

import torch
import torchvision.transforms as transforms
from torchvision.transforms import autoaugment

from typing import Tuple


def make_transforms(means: torch.Tensor, std_devs: torch.Tensor) -> Tuple:
    '''
    Given a tensor of computed means and a tensor of computed standard devations,
    return's a tuple containing a train and test transform pipelines
    '''
    train_transforms = transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomCrop(32, padding=2),
        transforms.ToTensor(),
        transforms.Normalize(mean=means,
                             std=std_devs)
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=means,
                             std=std_devs)
    ])

    return train_transforms, test_transforms


def make_auto_transforms(means: torch.Tensor, std_devs: torch.Tensor) -> Tuple:
    '''
    Utilizes PyTorch'es Auto

# Load Data

In [5]:
from src.data import get_transformed_data, make_data_loaders

# from src.transforms import make_transforms
from src.transforms import make_auto_transforms

In [6]:
BATCH_SIZE = 512
valid_ratio = 0.1

train_data, valid_data, test_data = (
    get_transformed_data(
        make_transforms = make_auto_transforms,
        valid_ratio = valid_ratio
    )
)

train_iterator, valid_iterator, test_iterator = (
    make_data_loaders(
        train_data,
        valid_data,
        test_data,
        batch_size=BATCH_SIZE,
    )
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
Files already downloaded and verified


# Use The Transformed Data To Train A Model

In [7]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

model_architecture = (
    (1, 128, 0.5),
    (2, 128, 0.5),
    (2, 128, 0.5),
    (2, 128, 0.5),
    (2, 196, 0.5),
    (2, 196, 0.5),
)

stem_config = StemConfig(num_channels=128, kernel_size=5, stride=1, padding=2)
model = ResNet(model_architecture, stem_config=stem_config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# pass a dummy tensor through the model,
# thus specifying the shape of nn.LazyCond2d and
# converting nn.LazyConv2d to nn.Conv2d
inputs = torch.empty((BATCH_SIZE, 3, 32, 32))
inputs.normal_()

model = model.to(device)

y = model(inputs.to(device))
print(y.size())

torch.Size([512, 10])


In [9]:
import torch.nn as nn

from src.engine import train_one_epoch, evaluate

EPOCHS  = 3
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
import time

best_loss = float('inf')
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

Epoch 1
	Train elapsed: 1:13, loss: 1.8592, acc: 29.88%
	Validation elapsed: 0:3, loss: 1.5745, acc: 41.44%
Epoch 2
	Train elapsed: 1:15, loss: 1.6375, acc: 39.35%
	Validation elapsed: 0:3, loss: 1.3838, acc: 49.06%
Epoch 3
	Train elapsed: 1:15, loss: 1.4697, acc: 46.12%
	Validation elapsed: 0:3, loss: 1.2370, acc: 56.07%
